#Some good tutorials

https://sci2lab.github.io/ml_tutorial/multiclass_classification/#Naive-Bayes-Classifier

In [1]:
import sklearn
sklearn_version = sklearn.__version__
print(sklearn_version)
#needs to be 0.24.2

import gensim
gensim.__version__
#4.0.1

0.24.1


/Users/carlyknight/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


'4.0.1'

In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#from gensim.models.deprecated.doc2vec import LabeledSentence
from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
from string import digits

import pandas as pd
import numpy as np
import string
import re
import random 
import os
import csv
import pickle

from sklearn import metrics
from sklearn import svm
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier 
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, plot_precision_recall_curve, auc, average_precision_score,classification_report, confusion_matrix, accuracy_score, average_precision_score, precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score, cross_validate, RepeatedStratifiedKFold, train_test_split,KFold, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn import preprocessing





from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from modAL.uncertainty import entropy_sampling
from modAL.density import information_density

from scipy.stats import entropy
from matplotlib import pyplot as plt
%matplotlib inline

from numpy import mean
from numpy import std

# Load Data and Clean

In [4]:
data1 = pd.read_csv("/Users/carlyknight/Dropbox/PROJECTS/Corporate Annual Reports/Replication Folder RR/dictionary_verification/training_paragraphlevel_8-30-21.csv")
data1['dataset'] = 1
data1.shape

(1000, 12)

In [5]:
data2 = pd.read_csv("/Users/carlyknight/Dropbox/PROJECTS/Corporate Annual Reports/Replication Folder RR/dictionary_verification/training_paragraphlevel_set2-9-19-21.csv")
data2['dataset'] = 2
data2.shape

(998, 14)

In [6]:
#combine datasets
data1 = data1[["dataset","text", "Coordination", "Cooperation"]]
data2 = data2[["dataset", "text", "Coordination", "Cooperation"]]
datafull = data1.append(data2)
datafull.shape

(1998, 4)

In [7]:
# preprocessing (remove punctuation, numbers, lowercase)/ remove stopwords
punctuation_dictionary = {s:None for s in list(string.punctuation)}

punctuation_translator = str.maketrans(punctuation_dictionary)

def text_cleaner(text, punctuation_translator):
    text=str(text)
    text = text.replace('c("', '')
    text = str(text).translate(punctuation_translator)
    text = text.lower()
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    text= remove_stopwords(text)
    return(text)

datafull["clean_text"] = datafull["text"].apply(lambda x: text_cleaner(x, punctuation_translator))

In [8]:
# find phrases
phrases1 = Phrases(map(lambda x: x.split(), datafull["clean_text"].tolist())) #bigram
phrases2 = Phrases(phrases1[map(lambda x: x.split(), datafull["clean_text"].tolist())]) #trigram
datafull["phrased_text"] = datafull["clean_text"].apply(lambda x: " ".join(phrases2[phrases1[x.split()]]))

#add row_id
datafull['row.id'] = np.arange(len(datafull))
datafull["phrased_text"].head

<bound method NDFrame.head of 0      atyred checchxcxac cochairman gary l wilson co...
1      management took actions year reduce costs cons...
2      finest fsolntcs gii iced service icljxcxac sei...
3      union_camp aggressively pursued forest managem...
4      stockholder rights plan pursuant stockholder r...
                             ...                        
993    tax consolidated return adjustments prior_year...
994    employees work rod wire division called work s...
995    armstrong world industries subsidiaries longte...
996    weve broken free traditional geographic approa...
997    july cpuc public staff recommended million set...
Name: phrased_text, Length: 1998, dtype: object>

# Combine cooperation & coordination

In [9]:
#drop data that has both coordination and cooperation
data_multi = datafull[~((datafull["Cooperation"] == 1) & (datafull["Coordination"] == 1))]

In [10]:
#create an multinomial outcome
data_multi['Outcome']=0
data_multi.loc[data_multi['Cooperation'] ==1, 'Outcome'] = 1
data_multi.loc[data_multi['Coordination'] ==1, 'Outcome'] = 2

<ipython-input-10-467582cf8949>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_multi['Outcome']=0
/Users/carlyknight/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/carlyknight/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [11]:
#Cooperaiton: 424/2000 (0.21)
#Coordination: (393/2000 (0.19)

data_multi['Outcome' ].value_counts()

0    1160
1     424
2     393
Name: Outcome, dtype: int64

In [12]:
data_multi[data_multi["dataset"] ==1]["Outcome"].value_counts()

0    588
1    201
2    190
Name: Outcome, dtype: int64

In [13]:
data_multi[data_multi["dataset"] ==2]["Outcome"].value_counts()

0    572
1    223
2    203
Name: Outcome, dtype: int64

In [90]:
#vectorization for full dataset
vectorizer = CountVectorizer(min_df=5)
X = vectorizer.fit_transform(data_multi["phrased_text"]).toarray()

tfidfconverter = TfidfTransformer()
X_tf = tfidfconverter.fit_transform(X).toarray()

y = np.array(data_multi['Outcome'])

#training set and scale
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size=0.2, stratify = y)
print('Training set Shape: {}  | Test set Shape: {}'.format(X_train.shape, X_test.shape))

Training set Shape: (1581, 3019)  | Test set Shape: (396, 3019)


In [15]:
#vectorization for just dataset 1
X1 = vectorizer.fit_transform(data_multi[data_multi["dataset"] ==1]["phrased_text"]).toarray()

tfidfconverter = TfidfTransformer()
X_tf1 = tfidfconverter.fit_transform(X1).toarray()

y1 = np.array(data_multi[data_multi["dataset"] ==1]['Outcome'])

#training set and scale
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_tf1, y1, test_size=0.2, stratify = y1)

In [16]:
#vectorization for just dataset 2
X2 = vectorizer.fit_transform(data_multi[data_multi["dataset"] ==2]["phrased_text"]).toarray()
X_tf2 = tfidfconverter.fit_transform(X2).toarray()

y2 = np.array(data_multi[data_multi["dataset"] ==2]['Outcome'])

#training set and scale
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_tf2, y2, test_size=0.2, stratify = y2)

# Naive Bayes Classifier: Very bad recall!
https://sci2lab.github.io/ml_tutorial/multiclass_classification/#Naive-Bayes-Classifier

In [151]:
nb_model = MultinomialNB().fit(X_train, y_train)

nb_predicted = nb_model.predict(X_test)

# creating a confusion matrix
print(metrics.classification_report(y_test, nb_predicted))

              precision    recall  f1-score   support

           0       0.65      0.97      0.78       232
           1       0.94      0.20      0.33        85
           2       0.65      0.25      0.36        79

    accuracy                           0.66       396
   macro avg       0.75      0.47      0.49       396
weighted avg       0.71      0.66      0.60       396



# Random Forest Classifier: Macro avg in 50s

In [152]:
# training a DescisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dtree_model = DecisionTreeClassifier().fit(X_train, y_train)
dtree_predictions = dtree_model.predict(X_test)
 
# creating a confusion matrix
print(metrics.classification_report(y_test, dtree_predictions))

              precision    recall  f1-score   support

           0       0.70      0.71      0.70       232
           1       0.47      0.53      0.50        85
           2       0.40      0.32      0.35        79

    accuracy                           0.59       396
   macro avg       0.52      0.52      0.52       396
weighted avg       0.59      0.59      0.59       396



# SVM : linear seems to perform best 

(macro avg 70 precision, 55 recall)

In [153]:
linear_ovr = svm.SVC(kernel='linear', C=1, decision_function_shape='ovr').fit(X_train, y_train)
print(metrics.classification_report(y_test, linear_ovr.predict(X_test)))

              precision    recall  f1-score   support

           0       0.70      0.92      0.79       232
           1       0.80      0.42      0.55        85
           2       0.57      0.33      0.42        79

    accuracy                           0.69       396
   macro avg       0.69      0.56      0.59       396
weighted avg       0.69      0.69      0.67       396



### dataset 1

In [154]:
linear_ovr = svm.SVC(kernel='linear', C=1, decision_function_shape='ovr').fit(X_train1, y_train1)
print(metrics.classification_report(y_test1, linear_ovr.predict(X_test1)))

              precision    recall  f1-score   support

           0       0.71      0.95      0.81       118
           1       0.72      0.33      0.45        40
           2       0.57      0.32      0.41        38

    accuracy                           0.70       196
   macro avg       0.67      0.53      0.56       196
weighted avg       0.69      0.70      0.66       196



In [155]:
rbf = svm.SVC(kernel='rbf', gamma=1, C=1, decision_function_shape='ovo').fit(X_train, y_train)
print(metrics.classification_report(y_test, rbf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.62      0.98      0.76       232
           1       0.90      0.21      0.34        85
           2       0.75      0.11      0.20        79

    accuracy                           0.64       396
   macro avg       0.76      0.43      0.43       396
weighted avg       0.71      0.64      0.56       396



In [156]:
sig = svm.SVC(kernel='sigmoid', C=1, decision_function_shape='ovo').fit(X_train, y_train)
print(metrics.classification_report(y_test, sig.predict(X_test)))

              precision    recall  f1-score   support

           0       0.70      0.93      0.80       232
           1       0.86      0.42      0.57        85
           2       0.57      0.33      0.42        79

    accuracy                           0.70       396
   macro avg       0.71      0.56      0.59       396
weighted avg       0.71      0.70      0.67       396



# Logistic Multiclass using OVR 

(macro avg 70 precision, mid50s recall)

In [92]:
model_lm = LogisticRegressionCV(multi_class='ovr', solver = 'liblinear')
model_lm.fit(X_train, y_train)
print(metrics.classification_report(y_test, model_lm.predict(X_test)))

              precision    recall  f1-score   support

           0       0.71      0.88      0.79       232
           1       0.72      0.48      0.58        85
           2       0.61      0.38      0.47        79

    accuracy                           0.70       396
   macro avg       0.68      0.58      0.61       396
weighted avg       0.69      0.70      0.68       396



## Using previously just dataset1

In [158]:
model_lm.fit(X_train1, y_train1)
print(metrics.classification_report(y_test1, model_lm.predict(X_test1)))

              precision    recall  f1-score   support

           0       0.70      0.92      0.79       118
           1       0.64      0.40      0.49        40
           2       0.50      0.21      0.30        38

    accuracy                           0.67       196
   macro avg       0.61      0.51      0.53       196
weighted avg       0.65      0.67      0.63       196



# Multiclass using multinomial

(about same as OVR)

In [159]:
model2 = LogisticRegressionCV(multi_class='multinomial', solver = 'newton-cg' )
model2.fit(X_train, y_train)
print(metrics.classification_report(y_test, model2.predict(X_test)))

              precision    recall  f1-score   support

           0       0.70      0.91      0.79       232
           1       0.75      0.48      0.59        85
           2       0.59      0.29      0.39        79

    accuracy                           0.69       396
   macro avg       0.68      0.56      0.59       396
weighted avg       0.69      0.69      0.66       396



In [160]:
#https://towardsdatascience.com/comprehensive-guide-on-multiclass-classification-metrics-af94cfb83fbd
from sklearn.metrics import roc_auc_score

# Generate class membership probabilities
y_preb_probs = model2.predict_proba(X_test)

roc_auc_score(y_test, y_preb_probs, average="weighted", multi_class="ovr")

0.8351610723179527

In [ ]:
#but is this good?
#https://inside.getyourguide.com/blog/2020/9/30/what-makes-a-good-f1-score


### Using  just dataset 1

In [161]:
model2.fit(X_train1, y_train1)
print(metrics.classification_report(y_test1, model2.predict(X_test1)))

              precision    recall  f1-score   support

           0       0.73      0.94      0.82       118
           1       0.72      0.33      0.45        40
           2       0.64      0.42      0.51        38

    accuracy                           0.71       196
   macro avg       0.70      0.56      0.59       196
weighted avg       0.71      0.71      0.68       196



### Using just dataset2

In [162]:
model2.fit(X_train2, y_train2)
print(metrics.classification_report(y_test2, model2.predict(X_test2)))

              precision    recall  f1-score   support

           0       0.71      0.89      0.79       114
           1       0.66      0.42      0.51        45
           2       0.67      0.44      0.53        41

    accuracy                           0.69       200
   macro avg       0.68      0.59      0.61       200
weighted avg       0.69      0.69      0.67       200



# Ok let's try BERT

#https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613

#https://medium.com/analytics-vidhya/multiclass-text-categorization-97-perc-accuracy-bert-model-2b97d8118903


In [3]:
import torch
from torch.utils.data import TensorDataset

from tqdm.notebook import tqdm

# Import tokenizer from transformers package
import transformers
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

transformers.__version__

#needs to be 4.11.3

'4.11.3'

# Prep data for Bert

In [17]:
data_multi['label'] = data_multi['Outcome']
data_multi['guid'] = data_multi['row.id']
data_multi['text_a'] = data_multi['clean_text']

<ipython-input-17-3a9c9970389c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_multi['label'] = data_multi['Outcome']
<ipython-input-17-3a9c9970389c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_multi['guid'] = data_multi['row.id']
<ipython-input-17-3a9c9970389c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [18]:
data_multi.columns

Index(['dataset', 'text', 'Coordination', 'Cooperation', 'clean_text',
       'phrased_text', 'row.id', 'Outcome', 'label', 'guid', 'text_a'],
      dtype='object')

In [19]:
#training set and scale
X_train, X_val, y_train, y_val = train_test_split(data_multi.index.values, data_multi.label.values, test_size=0.2, stratify =  data_multi.label.values)

data_multi['data_type'] = ['not_set']*data_multi.shape[0]
data_multi.loc[X_train, 'data_type'] = 'train'
data_multi.loc[X_val, 'data_type'] = 'val'

data_multi.groupby(['Outcome', 'label', 'data_type']).count()

<ipython-input-19-a50269c2b71e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_multi['data_type'] = ['not_set']*data_multi.shape[0]
/Users/carlyknight/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


dataset  text  Coordination  Cooperation  clean_text  \
Outcome label data_type                                                         
0       0     train          750   737           750          750         750   
              val            410   405           410          410         410   
1       1     train          279   279           279          279         279   
              val            145   145           145          145         145   
2       2     train          255   255           255          255         255   
              val            138   138           138          138         138   

                         phrased_text  row.id  guid  text_a  
Outcome label data_type                                      
0       0     train               750     750   750     750  
              val                 410     410   410     410  
1       1     train               279     279   279     279  
              val                 145     145   145     145  
2       2     train               255     255   255     255  
              val                 138     138   138     138

In [37]:
#https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613
# Load the tokenizer 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

encoded_data_train = tokenizer.batch_encode_plus(
    data_multi[data_multi.data_type=='train'].phrased_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    return_tensors='pt',
     max_length=512, 
    truncation = True
)

encoded_data_val = tokenizer.batch_encode_plus(
    data_multi[data_multi.data_type=='val'].phrased_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    truncation = True,
     max_length=512, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data_multi[data_multi.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data_multi[data_multi.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [29]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [30]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [31]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


In [33]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [38]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        batch = tuple(b.to(device) for b in batch) 
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    


cpu


In [39]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
             
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/428 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.8122250058766559
Validation loss: 0.7181156595031937
F1 Score (Weighted): 0.6533111891602458


Epoch 2:   0%|          | 0/428 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.5893202567098784
Validation loss: 0.8510730570561184
F1 Score (Weighted): 0.7300694117038976


...

Epoch 4:   0%|          | 0/428 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.4039506095523563
Validation loss: 1.0775867314027519
F1 Score (Weighted): 0.7591156679334382


Epoch 5:   0%|          | 0/428 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.31343804409827547


KeyboardInterrupt: 

In [147]:
preds_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_vals.flatten()

counts = Counter(zip(preds_flat, labels_flat))

#Total Predicted
predicted_0 = len(preds_flat[preds_flat==0])
predicted_1 = len(preds_flat[preds_flat==1])
predicted_2 = len(preds_flat[preds_flat==2])

#Total Gold
gold_0 = len(labels_flat[labels_flat==0])
gold_1 = len(labels_flat[labels_flat==1])
gold_2 = len(labels_flat[labels_flat==2])

#Class 0
print("CLASS 0:")
print(f'Recall Class 0: { counts[0, 0]/ gold_0}') 
print(f'Precision Class 0: { counts[0, 0]/ predicted_0}\n') 

#Class 1
print("CLASS 1:")
print(f'Recall Class 1: { counts[1, 1]/ gold_1}') 
print(f'Precision Class 1: { counts[1, 1]/ predicted_1}\n') 

#Class 2
print("CLASS 2:")
print(f'Recall Class 2: { counts[2, 2]/ gold_2}') 
print(f'Precision Class 2: { counts[2, 2]/ predicted_2}\n ') 


CLASS 0:
Recall Class 0: 0.8341463414634146
Precision Class 0: 0.8592964824120602

CLASS 1:
Recall Class 1: 0.5862068965517241
Precision Class 1: 0.648854961832061

CLASS 2:
Recall Class 2: 0.6811594202898551
Precision Class 2: 0.573170731707317
 


In [47]:
#save model
torch.save(model.state_dict(), f'/Users/carlyknight/Dropbox/PROJECTS/Corporate Annual Reports/Replication Folder RR/dictionary_verification/finetuned_BERT_epoch_5.model')  

# Re-estimate

In [149]:
modelBert5 = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

modelBert5.to(device)

modelBert5.load_state_dict(torch.load('/Users/carlyknight/Dropbox/PROJECTS/Corporate Annual Reports/Replication Folder RR/dictionary_verification/finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

_, predictions1, true_vals1 = evaluate(dataloader_validation)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [150]:
preds_flat = np.argmax(predictions1, axis=1).flatten()
labels_flat = true_vals1.flatten()

counts = Counter(zip(preds_flat, labels_flat))

#Total Predicted
predicted_0 = len(preds_flat[preds_flat==0])
predicted_1 = len(preds_flat[preds_flat==1])
predicted_2 = len(preds_flat[preds_flat==2])

#Total Gold
gold_0 = len(labels_flat[labels_flat==0])
gold_1 = len(labels_flat[labels_flat==1])
gold_2 = len(labels_flat[labels_flat==2])

#Class 0
print("CLASS 0:")
print(f'Recall Class 0: { counts[0, 0]/ gold_0}') 
print(f'Precision Class 0: { counts[0, 0]/ predicted_0}\n') 

#Class 1
print("CLASS 1:")
print(f'Recall Class 1: { counts[1, 1]/ gold_1}') 
print(f'Precision Class 1: { counts[1, 1]/ predicted_1}\n') 

#Class 2
print("CLASS 2:")
print(f'Recall Class 2: { counts[2, 2]/ gold_2}') 
print(f'Precision Class 2: { counts[2, 2]/ predicted_2}\n ') 



CLASS 0:
Recall Class 0: 0.8341463414634146
Precision Class 0: 0.8592964824120602

CLASS 1:
Recall Class 1: 0.5862068965517241
Precision Class 1: 0.648854961832061

CLASS 2:
Recall Class 2: 0.6811594202898551
Precision Class 2: 0.573170731707317
 
